I created 2 Programs , 1 For ML and 1 for DL in order to compare the result between both and confirm which one is better
1) I chose HOG not SIFT because it looks at all image while SIFT processes only keypoints. For Leaves Classification is processing all the image crucial
2) I chose GB instead of RF because its efficienter since every estimator corrects the mistake of the previous one in series
3) Program Overview: Load Train and Test CSV data (TO SHOW) -> extract features of images -> train and validation -> Testing (Prediction) -> Printing the result

In [12]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import xgboost as xgb
from skimage.feature import hog
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.metrics')


In [13]:
# Configuration
BASE_PATH = "C:/Users/Sami/Desktop/CV"
IMAGE_PATH = os.path.join(BASE_PATH, "images")
TRAIN_CSV = os.path.join(BASE_PATH, "train.csv")
TEST_CSV = os.path.join(BASE_PATH, "test.csv")
SUBMISSION_CSV = "submission_xgboost.csv"
IMAGE_SIZE = 128



I defined the function HOG to extract features

In [ ]:
# Define function to extract HOG features from a given image path
def extract_hog_features(img_path):
    
        img = Image.open(img_path).convert('L') # Load image and convert to grayscale
        img = img.resize((IMAGE_SIZE, IMAGE_SIZE)) # Resize image to fixed size
        img = np.array(img) / 255.0
        features = hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2), 
                       orientations=9, visualize=False, feature_vector=True) # Extract HOG descriptor
        return features  # Return the feature vector ( HOG Descriptor ) # IMPORTANT!!!:




I loaded data from CSV data to variables (as Panda List) then encoded species to numbers for the model

In [ ]:
# Load Data
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
le = LabelEncoder()
train_df["species_encoded"] = le.fit_transform(train_df["species"])   # IMPORTANT!!!: encoding labels to numbers for the model
NUM_CLASSES = len(le.classes_)

print(f"Classes: {NUM_CLASSES}, Training: {len(train_df)}, Test: {len(test_df)}")



Classes: 99, Training: 990, Test: 594


Extract HOG Features with extract_hog_features function I defined and converted the panda lists to numpy arrays which work with the model of ML

In [ ]:
# Extract HOG Features
X_train_full = []
y_train_full = []

for idx,row in tqdm(train_df.iterrows(), total=len(train_df)): # Loop over each training row with progress bar
    img_path = os.path.join(IMAGE_PATH, f"{int(row['id'])}.jpg") # Construct image filepath
    features = extract_hog_features(img_path) # Extract HOG features
    X_train_full.append(features) # Append features to list
    y_train_full.append(row["species_encoded"]) # Append label list.
    
    
# Convert to numpy array for MODEL INPUT
X_train_full = np.array(X_train_full)
y_train_full = np.array(y_train_full)
print(f"Feature shape: {X_train_full.shape}") # (num_images, num_features)



100%|██████████| 990/990 [00:02<00:00, 342.85it/s]

Feature shape: (990, 1764)


Train/Validation Split

In [17]:
# Train/Validation Split
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.15, 
    stratify=y_train_full, random_state=42
)



Train the Model with XGB Classifier

In [ ]:
# Train XGBoost
model = xgb.XGBClassifier(
    n_estimators=250,      # IMPORTANT!!!:     
    max_depth=50,             
    learning_rate=0.025,     # IMPORTANT!!!:     
    subsample=0.8, #80% of data to train each tree
    colsample_bytree=0.8, # 80% of features to train each tree
    random_state=42
)

model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=10)


[0]	validation_0-mlogloss:4.54662
[10]	validation_0-mlogloss:4.13869
[20]	validation_0-mlogloss:3.78195
[30]	validation_0-mlogloss:3.47670
[40]	validation_0-mlogloss:3.24382
[50]	validation_0-mlogloss:3.04523
[60]	validation_0-mlogloss:2.87827
[70]	validation_0-mlogloss:2.73846
[80]	validation_0-mlogloss:2.62405
[90]	validation_0-mlogloss:2.52275
[100]	validation_0-mlogloss:2.43116
[110]	validation_0-mlogloss:2.35404
[120]	validation_0-mlogloss:2.28268
[130]	validation_0-mlogloss:2.22009
[140]	validation_0-mlogloss:2.16216
[150]	validation_0-mlogloss:2.11125
[160]	validation_0-mlogloss:2.06027
[170]	validation_0-mlogloss:2.01328
[180]	validation_0-mlogloss:1.96924
[190]	validation_0-mlogloss:1.93039
[200]	validation_0-mlogloss:1.89357
[210]	validation_0-mlogloss:1.86107
[220]	validation_0-mlogloss:1.82540
[230]	validation_0-mlogloss:1.79636
[240]	validation_0-mlogloss:1.77020
[249]	validation_0-mlogloss:1.74591


,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [19]:
# Evaluate Model
y_val_pred = model.predict(X_val)
val_acc = accuracy_score(y_val, y_val_pred) * 100
print(f"Validation Accuracy: {val_acc:.2f}%")



Validation Accuracy: 71.14%


I extracted features of all the 1600 Images

In [20]:
# Extract HOG Features for All Test Images ( Test Predictions )
X_test = []
for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    img_path = os.path.join(IMAGE_PATH, f"{int(row['id'])}.jpg")
    features = extract_hog_features(img_path)
    X_test.append(features)

X_test = np.array(X_test) 
test_probs = model.predict_proba(X_test) # Predict class/Label probabilities according to features



100%|██████████| 594/594 [00:01<00:00, 389.64it/s]


Print of submission data with ID of images and its possible label found
See example of Image 16,19

In [21]:
# Index of class with highest probability for each image
y_test_pred_idx = test_probs.argmax(axis=1) 

# Convert class indices back to species names
y_test_pred_labels = le.inverse_transform(y_test_pred_idx)

# Build submission table: id + predicted species
submission = pd.DataFrame({
    "id": test_df["id"].astype(int),
    "species": y_test_pred_labels
})

#Sort by id
submission = submission.sort_values("id").reset_index(drop=True)

# Save CSV
submission.to_csv("submission_GB_HOG.csv", index=False)
print("Saved as submission_GB_HOG.csv")




Saved as submission_GB_HOG.csv
